<a href="https://colab.research.google.com/github/fzantalis/colab_collection/blob/master/ttmai_image_to_painting_prog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Αυτό το notebook είναι ένα demo της τεχνικής "Stylized Neural Painting, arXiv:2011.08114.". 

Εδώ μπορούμε να δημιουργήσουμε ένα βίντεο με μια εικόνα που ζωγραφίζεται σταδιακά από ένα Τεχνητό Νευρωνικό Δίκτυο.

Μπορείτε να βρείτε περισσότερες πληροφορίες στους παρακάτω συνδέσμους:
 
[Project Page](https://jiupinjia.github.io/neuralpainter/) | [GitHub](https://github.com/jiupinjia/stylized-neural-painting) | [Preprint](https://arxiv.org/abs/2011.08114)

<a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" /></a><span xmlns:dct="http://purl.org/dc/terms/" property="dct:title">   The project </a> is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License</a>.

# 1. Κατεβάζουμε το repository από το github

In [ ]:
# Clone the repository
!git clone https://github.com/jiupinjia/stylized-neural-painting

Μπαίνουμε στο φάκελο του project

In [ ]:
cd stylized-neural-painting

# 2. Φορτώνουμε τις βιβλιοθήκες


In [ ]:
import argparse
 
import torch
torch.cuda.current_device()
import torch.optim as optim
 
from painter import *
# Decide which device we want to run on
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# 3. Ανεβάζουμε την εικόνα μας

In [ ]:
from google.colab import files 
uploaded = files.upload() 
for img in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=img, length=len(uploaded[img])))

# 4. Ρυθμίζουμε τις παραμέτρους του Script

In [ ]:
# settings
parser = argparse.ArgumentParser(description='STYLIZED NEURAL PAINTING')
args = parser.parse_args(args=[])
args.img_path = '/content/stylized-neural-painting/'+img # path to input photo
args.renderer = 'oilpaintbrush' # [watercolor, markerpen, oilpaintbrush, rectangle]
args.canvas_color = 'black' # [black, white]
args.canvas_size = 512 # size of the canvas for stroke rendering'
args.keep_aspect_ratio = False # whether to keep input aspect ratio when saving outputs
args.max_m_strokes = 500 # max number of strokes
args.max_divide = 5 # divide an image up-to max_divide x max_divide patches
args.beta_L1 = 1.0 # weight for L1 loss
args.with_ot_loss = False # set True for imporving the convergence by using optimal transportation loss, but will slow-down the speed
args.beta_ot = 0.1 # weight for optimal transportation loss
args.net_G = 'zou-fusion-net' # renderer architecture
args.renderer_checkpoint_dir = './checkpoints_G_oilpaintbrush' # dir to load the pretrained neu-renderer
args.lr = 0.005 # learning rate for stroke searching
args.output_dir = './output' # dir to save painting results
args.disable_preview = True # disable cv2.imshow, for running remotely without x-display

# 4. Κατεβάζουμε το προ-εκπαιδευμένο μοντέλο

In [ ]:
# Define some helper functions for downloading pretrained model
# taken from this StackOverflow answer: https://stackoverflow.com/a/39225039
import requests
 
def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"
 
    session = requests.Session()
 
    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)
 
    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)
 
    save_response_content(response, destination)    
 
def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value
 
    return None
 
def save_response_content(response, destination):
    CHUNK_SIZE = 32768
 
    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

In [ ]:
# download and unzip...
file_id = '1sqWhgBKqaBJggl2A8sD1bLSq2_B1ScMG'
destination = './checkpoints_G_oilpaintbrush.zip'
download_file_from_google_drive(file_id, destination)

In [ ]:
!unzip checkpoints_G_oilpaintbrush.zip

# 5. Ορίζουμε την βασική συνάρτηση του προγράμματος

In [ ]:
def optimize_x(pt):
 
    pt._load_checkpoint()
    pt.net_G.eval()
 
    print('begin drawing...')
 
    PARAMS = np.zeros([1, 0, pt.rderr.d], np.float32)
 
    if pt.rderr.canvas_color == 'white':
        CANVAS_tmp = torch.ones([1, 3, 128, 128]).to(device)
    else:
        CANVAS_tmp = torch.zeros([1, 3, 128, 128]).to(device)
 
    for pt.m_grid in range(1, pt.max_divide + 1):
 
        pt.img_batch = utils.img2patches(pt.img_, pt.m_grid, pt.net_G.out_size).to(device)
        pt.G_final_pred_canvas = CANVAS_tmp
 
        pt.initialize_params()
        pt.x_ctt.requires_grad = True
        pt.x_color.requires_grad = True
        pt.x_alpha.requires_grad = True
        utils.set_requires_grad(pt.net_G, False)
 
        pt.optimizer_x = optim.RMSprop([pt.x_ctt, pt.x_color, pt.x_alpha], lr=pt.lr, centered=True)
 
        pt.step_id = 0
        for pt.anchor_id in range(0, pt.m_strokes_per_block):
            pt.stroke_sampler(pt.anchor_id)
            iters_per_stroke = int(500 / pt.m_strokes_per_block)
            for i in range(iters_per_stroke):
                pt.G_pred_canvas = CANVAS_tmp
 
                # update x
                pt.optimizer_x.zero_grad()
 
                pt.x_ctt.data = torch.clamp(pt.x_ctt.data, 0.1, 1 - 0.1)
                pt.x_color.data = torch.clamp(pt.x_color.data, 0, 1)
                pt.x_alpha.data = torch.clamp(pt.x_alpha.data, 0, 1)
 
                pt._forward_pass()
                pt._drawing_step_states()
                pt._backward_x()
 
                pt.x_ctt.data = torch.clamp(pt.x_ctt.data, 0.1, 1 - 0.1)
                pt.x_color.data = torch.clamp(pt.x_color.data, 0, 1)
                pt.x_alpha.data = torch.clamp(pt.x_alpha.data, 0, 1)
 
                pt.optimizer_x.step()
                pt.step_id += 1
 
        v = pt._normalize_strokes(pt.x)
        v = pt._shuffle_strokes_and_reshape(v)
        PARAMS = np.concatenate([PARAMS, v], axis=1)
        CANVAS_tmp = pt._render(PARAMS, save_jpgs=False, save_video=False)
        CANVAS_tmp = utils.img2patches(CANVAS_tmp, pt.m_grid + 1, pt.net_G.out_size).to(device)
 
    pt._save_stroke_params(PARAMS)
    final_rendered_image = pt._render(PARAMS, save_jpgs=False, save_video=True)
 
    return final_rendered_image

# 6. Ξεκινάμε την ζωγραφική!

In [ ]:
pt = ProgressivePainter(args=args)
final_rendered_image = optimize_x(pt)

# 7. Προβολή της τελικής ζωγραφισμένης εικόνας

In [ ]:
plt.imshow(final_rendered_image), plt.title('generated')
plt.show()

# 8. Προβολή του τελικού βίντεο 

In [ ]:
from IPython.display import HTML
from base64 import b64encode
import os

img_name = img.split('.')[0]
source_vid = '/content/stylized-neural-painting/output/'+img_name+'_animated.mp4'
compressed_vid = '/content/stylized-neural-painting/output/'+img_name+'_compressed.mp4'
os.system(f"ffmpeg -i {source_vid} -vcodec libx264 {compressed_vid}")

mp4 = open(compressed_vid,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)